In [1]:
# Create a function to setup the directories we'll be storing our images
def makedir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
        return None
    else:
        pass

In [2]:
num_classes=5
num_train_images = 500
num_test_images = 200

In [3]:
import cv2
import os
import numpy as np
import pandas as pd


train = np.random.randn(num_classes,num_train_images,28,28)
test  = np.random.randn(num_classes,num_test_images,28,28)


cap = cv2.VideoCapture(0)
for i in range(0,num_classes):
    if i == 0:
        while cv2.waitKey(1) != 13:
            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)

            #define region of interest
            roi = frame[100:400, 320:620]
            cv2.imshow('roi', roi)
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            roi = cv2.resize(roi, (28, 28), interpolation = cv2.INTER_AREA)

            cv2.imshow('roi sacled and gray', roi)
            copy = frame.copy()
            cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)

            cv2.putText(copy, "Hit Enter to Record when ready", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
            cv2.imshow('frame', copy)
        image_count = 0
    
    
    
    while(cv2.waitKey(1) != 13 and image_count!=num_train_images):
        
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        #define region of interest
        roi = frame[100:400, 320:620]
        cv2.imshow('roi', roi)
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi = cv2.resize(roi, (28, 28), interpolation = cv2.INTER_AREA)

        cv2.imshow('roi sacled and gray', roi)
        copy = frame.copy()
        cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)

        
        cv2.putText(copy, "Recording the "+str(i+1)+" gesture - Train", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.imshow('frame', copy)
        gesture_dir = './handgestures/train/'+str(i)+'/'
        makedir(gesture_dir)
        cv2.imwrite(gesture_dir + str(image_count) + ".jpg", roi)
        
        train[i][image_count] = cv2.imread(gesture_dir + str(image_count) + ".jpg",cv2.IMREAD_GRAYSCALE)
        
        image_count+=1
        
    image_count = 0
    
    
    
    while(cv2.waitKey(1) != 13 and image_count!=num_test_images):
        
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)

        #define region of interest
        roi = frame[100:400, 320:620]
        cv2.imshow('roi', roi)
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        roi = cv2.resize(roi, (28, 28), interpolation = cv2.INTER_AREA)

        cv2.imshow('roi sacled and gray', roi)
        copy = frame.copy()
        cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)

        cv2.putText(copy, "Recording the "+str(i+1)+" gesture - Test", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.putText(copy, str(image_count), (400 , 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
        cv2.imshow('frame', copy)
        gesture_dir = './handgestures/test/'+str(i)+'/'
        makedir(gesture_dir)
        cv2.imwrite(gesture_dir + str(image_count) + ".jpg", roi)
        
        test[i][image_count] = cv2.imread(gesture_dir + str(image_count) + ".jpg",cv2.IMREAD_GRAYSCALE)
        
        image_count+=1
        
    image_count = 0
    
    
    
    if i != num_classes-1:
        while(cv2.waitKey(1) != 13):
        
            ret, frame = cap.read()
            frame = cv2.flip(frame, 1)

            #define region of interest
            roi = frame[100:400, 320:620]
            cv2.imshow('roi', roi)
            roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            roi = cv2.resize(roi, (28, 28), interpolation = cv2.INTER_AREA)

            cv2.imshow('roi sacled and gray', roi)
            copy = frame.copy()
            cv2.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)

            cv2.putText(copy, "Click enter to continue ", (100 , 100), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
            cv2.imshow('frame', copy)




    
while cv2.waitKey(1) != 13:
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    copy = frame.copy()
    cv2.putText(copy, "Hit Enter to Exit", (100 , 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    cv2.imshow('frame', copy)

cap.release()
cv2.destroyAllWindows() 

In [4]:
print(train.shape)
print(test.shape)

(5, 500, 28, 28)
(5, 200, 28, 28)


In [5]:
train_data=np.random.randn(num_classes*num_train_images,784)
test_data=np.random.randn(num_classes*num_test_images,784)
label_train=[]
label_test=[]
for i in range(0,num_classes):
    for j in range(0,num_train_images):
        label_train.append(i)
for i in range(0,num_classes):
    for j in range(0,num_test_images):
        label_test.append(i)

In [6]:
for c in range(0,num_classes):
    train_data[c*num_train_images:num_train_images*(c+1)] = np.array([i.flatten() for i in train[c]])
    test_data[c*num_test_images:num_test_images*(c+1)] = np.array([i.flatten() for i in test[c]])

In [7]:
columns=[]
for i in range(0,784):
    columns.append('pixel'+str(i))

In [8]:
dataframe_train = pd.DataFrame(train_data,index=None,columns=columns)
dataframe_test=pd.DataFrame(test_data,index=None,columns=columns)
dataframe_train.insert(0, 'label', label_train)
dataframe_test.insert(0, 'label', label_test)

In [9]:
dataframe_train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,210.0,210.0,211.0,212.0,209.0,215.0,213.0,213.0,214.0,...,206.0,207.0,209.0,211.0,211.0,211.0,212.0,213.0,214.0,214.0
1,0,210.0,210.0,209.0,212.0,213.0,210.0,214.0,214.0,214.0,...,206.0,207.0,209.0,210.0,210.0,212.0,212.0,213.0,214.0,214.0
2,0,210.0,211.0,208.0,214.0,212.0,210.0,216.0,213.0,215.0,...,207.0,209.0,209.0,210.0,211.0,211.0,212.0,212.0,213.0,214.0
3,0,209.0,210.0,211.0,211.0,215.0,210.0,214.0,214.0,215.0,...,206.0,207.0,210.0,211.0,210.0,211.0,212.0,213.0,214.0,214.0
4,0,210.0,208.0,210.0,214.0,213.0,211.0,213.0,214.0,215.0,...,207.0,207.0,210.0,210.0,210.0,211.0,213.0,213.0,214.0,214.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,4,208.0,208.0,211.0,210.0,210.0,213.0,212.0,214.0,213.0,...,204.0,206.0,207.0,208.0,210.0,209.0,210.0,211.0,212.0,213.0
2496,4,210.0,208.0,210.0,209.0,211.0,211.0,211.0,215.0,214.0,...,209.0,206.0,207.0,209.0,210.0,209.0,211.0,212.0,212.0,212.0
2497,4,207.0,208.0,207.0,209.0,212.0,213.0,211.0,213.0,213.0,...,203.0,205.0,207.0,207.0,209.0,212.0,211.0,212.0,213.0,212.0
2498,4,209.0,209.0,208.0,209.0,212.0,211.0,210.0,213.0,213.0,...,196.0,206.0,207.0,208.0,209.0,210.0,210.0,211.0,212.0,212.0


In [10]:
dataframe_test

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,203.0,204.0,204.0,205.0,206.0,207.0,209.0,210.0,209.0,...,122.0,145.0,157.0,165.0,184.0,199.0,204.0,205.0,207.0,208.0
1,0,203.0,204.0,204.0,205.0,206.0,207.0,208.0,209.0,210.0,...,123.0,137.0,151.0,157.0,178.0,199.0,204.0,205.0,206.0,207.0
2,0,203.0,203.0,204.0,205.0,205.0,206.0,208.0,209.0,209.0,...,126.0,139.0,149.0,158.0,175.0,198.0,203.0,205.0,207.0,208.0
3,0,203.0,204.0,204.0,205.0,206.0,206.0,208.0,209.0,208.0,...,131.0,136.0,147.0,167.0,191.0,202.0,203.0,205.0,207.0,208.0
4,0,203.0,204.0,205.0,205.0,206.0,206.0,207.0,208.0,210.0,...,132.0,142.0,164.0,186.0,200.0,202.0,205.0,206.0,207.0,207.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,4,204.0,204.0,205.0,206.0,206.0,207.0,208.0,208.0,210.0,...,151.0,169.0,182.0,195.0,202.0,204.0,206.0,207.0,208.0,208.0
996,4,204.0,204.0,205.0,207.0,208.0,208.0,208.0,209.0,208.0,...,141.0,161.0,177.0,188.0,200.0,204.0,206.0,207.0,207.0,208.0
997,4,204.0,204.0,205.0,205.0,205.0,207.0,208.0,210.0,210.0,...,131.0,150.0,168.0,181.0,194.0,203.0,206.0,207.0,208.0,208.0
998,4,202.0,203.0,204.0,205.0,206.0,206.0,207.0,207.0,211.0,...,130.0,146.0,166.0,178.0,193.0,204.0,205.0,206.0,207.0,207.0


In [11]:
dataframe_train.to_csv(r'train_data.csv',index=False)
dataframe_test.to_csv(r'test_data.csv',index=False)